##Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##ResNet Implementation

### Load Dataset


torchvision 패키지에서 제공하는 STL-10

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
import os

path2data = '/data'

if  not os.path.exists(path2data):
  os.mkdir(path2data)

data_transformer = transforms.Compose([transforms.ToTensor()])
train_ds = datasets.STL10(path2data, split='train', download='True', transform=data_transformer)
valid_ds = datasets.STL10(path2data, split='test', download='True', transform=data_transformer)

print(train_ds.data.shape)
print(valid_ds.data.shape)